In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.convolution import Tophat2DKernel, CustomKernel
import ashd

In [2]:
dwarfs = Table.read('../data/nearby-dwarfs.fits')
dwarfs.rename_column('_RAJ2000', 'ra')
dwarfs.rename_column('_DEJ2000', 'dec')
dwarfs = dwarfs.to_pandas()
cut = (dwarfs['R1']>8) & (dwarfs['R1']<50) 

In [3]:
names, mu0, rad, ra_vals, dec_vals = dwarfs.loc[cut, ['Name', 'muV', 'R1', 'ra', 'dec']].T.values

In [5]:
%%time
if False:
    cats = []
    for name in names:
        print('running on', name, 'field')
        fetch = dwarfs['Name']==name
        ra, dec = coords = dwarfs.loc[fetch, ['ra', 'dec']].values[0]
        params = ashd.PipeParams()
        params.do_ring_filter = True
        params.r_inner = 5
        params.r_outer = 8
        #params.kernel = CustomKernel(exp_kern(5, 31))
        params.kernel = Tophat2DKernel(15)
        params.bh = 128
        params.bw = 128
        params.segmentation_map = True
        pipe = ashd.ASHDPipe(ra, dec, params=params)
        pipe.detect()
        pipe.calc_auto_params()
        pipe.display_sources(True, figsize=(15, 15), 
                             save_fn='/Users/protostar/Desktop/asas-sn-hd-figs/'+name+'.png')
        cats.append(pipe.sources)   

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


In [4]:
fetch = dwarfs['Name']=='Fornax'
ra, dec = coords = dwarfs.loc[fetch, ['ra', 'dec']].values[0]
params = ashd.PipeParams()
params.do_ring_filter = True
params.r_inner = 5
params.r_outer = 8
#params.kernel = CustomKernel(exp_kern(5, 31))
params.kernel = Tophat2DKernel(15)
params.bh = 128
params.bw = 128
params.segmentation_map = True
pipe = ashd.ASHDPipe(ra, dec, params=params)
pipe.detect()
pipe.calc_auto_params()

pipeline 24 | 2017-08-30 18:02:32,762 |  | INFO: fetching image nearest to ra, dec = 39.9971, -34.4492
pipeline 45 | 2017-08-30 18:02:35,006 |  | INFO: smoothing image with ring filter with r_in = 5 and r_out = 8
pipeline 51 | 2017-08-30 18:02:41,140 |  | INFO: measuring and subtracting background
pipeline 54 | 2017-08-30 18:02:41,210 |  | INFO: detecting sources


In [5]:
engine = ashd.database.connect('/Users/protostar/Desktop/test.db')
session = ashd.database.Session()

In [6]:
pipe.write_to_db(session)